# Fine-tuning : RF-DETR

Objectif : faire de RF-DETR un détecteur de poissons sur des images de 1920\*1080 pixels

In [2]:
# Importations
import os
import numpy as np
import torch
import torchvision
from torchvision import transforms
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import json
import cv2
import matplotlib.pyplot as plt

# Vérification de la disponibilité du GPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Device utilisé :", device)


Device utilisé : cuda
